In [2]:
import numpy as np
import random
import os


In [3]:
def nan_check(data, label):
    """Find out the rows in datasets and delete these rows
    
    """
    
    nan_rows = np.array(0); #define an array containg the no. of rows having 'nan'
    
    #collect all the numbers of 'nan'-data rows
    for i in range(len(data)):
        for j in range(16):
            if str(data[i][j]) == 'nan':
                nan_rows = np.append(nan_rows, i)
    nan_rows = np.delete(nan_rows, 0) #delete the first element of nan_rows which was made to fit the append()
    
    #output the dataset whose 'nan'-data rows have been deleted
    return np.delete(data, nan_rows, 0), np.delete(label, nan_rows, 0) 

In [4]:
def shuffle(data_set, label_set):
    """Randomly shuffle the data and label
    
    data_set    the data samples
    
    label_set   the lables
    """
    
    shuffled_data = np.zeros((data_set.shape))
    shuffled_label = np.zeros((label_set.shape))
    idx = np.array(xrange(len(label_set)))
    random.shuffle(idx)
    i = 0
    for j in idx:
        shuffled_data[i] = data_set[int(j)]
        shuffled_label[i] = label_set[int(j)]
        i += 1
    return shuffled_data, shuffled_label

In [5]:
def get_data(set_type):
    """Get data from files and storage them in a array. Return the data_set and label_set.
    
    set_type    the type of data set you want to build, including train dataset, dev dataset 
                and eval dataset
    """
    
    data_path = {'train': 'train/lab/hw1train_labels.txt', 'dev': 'dev/lab/hw1dev_labels.txt', \
                 'eval': 'eval/lab/hw1eval_labels.txt'} 

    label_array = np.loadtxt(data_path[set_type], dtype='string') #load the label file into a array

    #creat empty arrays to insert label and data
    label_set = np.zeros([len(label_array), 1])
    data_set = np.zeros([len(label_array), 16])
    
    # the first column of the label file is the label,
    # the second column is the corresbonding data file nam
    for i in range(len(label_array)): 
        #build the label set
        label_set[i] = label_array[i][0] # insert label into label_set
        
        #build the data set
        with open(label_array[i][1]) as data_file:
            data = data_file.readlines()[0].split() #find the data accoding to label
        for j in range(len(data)):
            data_set[i][j] = data[j] #insert data into the dataset
            
    data_set, label_set = nan_check(data_set, label_set) #delete the rows containing 'nan'

    return shuffle(data_set, label_set) #return the shuffled data set and label set

In [6]:
def linear_regression_gradient(data, label, weight, b):
    """Calculate the gradient of linear regression classifier. Return the gradient.
    
    """

    gradient_w, gradient_b = 0, 0
    for i in range(len(label)):
        gradient_w += (-2) * (label[i] - (np.dot(weight, data[i]) + b)) * data[i]
        gradient_b += (-2) * (label[i] - (np.dot(weight, data[i]) + b))

    return gradient_w, gradient_b

In [7]:
def logistic_regression_gradient(data, label, weight, b):
    """Calculate the gradient of logistic regression . Return the gradient
    
    """
    
    gradient_w, gradient_b = 0, 0
    for i in range(len(label)):
        gradient_w += (-2) * ((np.dot(weight, data[i]) + b) - label[i]) * (np.dot(weight, data[i]) + b) * \
                   (1 - (np.dot(weight, data[i]) + b)) * data[i]
        gradient_b += (-2) * ((np.dot(weight, data[i]) + b) - label[i]) * (np.dot(weight, data[i]) + b) * \
                   (1 - (np.dot(weight, data[i]) + b))
        
    return gradient_w / float(len(label)), gradient_b / float(len(label))

In [8]:
def gradient_descent(weight, b, learning_rate, gradient_w, gradient_b):
    """Update and return weight and b.
    
    """
    
    weight -= learning_rate * gradient_w
    b -= learning_rate * gradient_b
    return weight, b

In [9]:
def compute_MSE(data, label, weight, b, mse):
    """Compute the Mean Square Error
    
    """
    
    for i in range(len(label)):
        mse += (label[i] - (np.dot(weight, data[i]) + b)) ** 2
        
    mse = mse / len(label)
    return mse

In [10]:
def compute_mse(dev_data, dev_label, w, b):
    """Compute the mean square error
    
    """
    
    mse = 0
    mse = compute_MSE(dev_data, dev_label, w, b, mse)
    
    return mse

In [11]:
def compute_acc(data, label, w, b):
    """accuracy
    
    """
    
    acc = 0
    for i in range(len(label)):
        if label[i] == round(np.dot(w, data[i]) + b):
            acc += 1
    return acc / float(len(label))

In [12]:
def activate(epoch = 1000, lr = 0.00001):
    """
    
    """

    # data and parameter initialization
    w = 2 * np.random.random(size = 16) - 1
    b = 0

    train_data, train_label = get_data('train') #build the dataset for training network
    dev_data, dev_label = get_data('dev')
    
    for i in range(epoch):    
        g_w, g_b = logistic_regression_gradient(train_data, train_label, w, b)
        w, b = gradient_descent(w, b, lr, g_w, g_b)
    
        mse = compute_mse(dev_data, dev_label, w, b)
        acc = compute_acc(dev_data, dev_label, w, b)
        
        print "epoch %d, loss: %f, error rate: %s " % (i, mse, 1 - acc)

In [14]:
activate(1000, 0.0001)

epoch 0, loss: 9.979371, error rate: 0.867441860465 
epoch 1, loss: 8.903113, error rate: 0.858914728682 
epoch 2, loss: 8.243693, error rate: 0.846511627907 
epoch 3, loss: 7.785918, error rate: 0.848837209302 
epoch 4, loss: 7.442162, error rate: 0.841085271318 
epoch 5, loss: 7.169404, error rate: 0.841860465116 
epoch 6, loss: 6.943941, error rate: 0.839534883721 
epoch 7, loss: 6.751643, error rate: 0.840310077519 
epoch 8, loss: 6.583572, error rate: 0.840310077519 
epoch 9, loss: 6.433819, error rate: 0.841085271318 
epoch 10, loss: 6.298325, error rate: 0.838759689922 
epoch 11, loss: 6.174219, error rate: 0.839534883721 
epoch 12, loss: 6.059413, error rate: 0.837984496124 
epoch 13, loss: 5.952355, error rate: 0.837984496124 
epoch 14, loss: 5.851866, error rate: 0.840310077519 
epoch 15, loss: 5.757033, error rate: 0.839534883721 
epoch 16, loss: 5.667137, error rate: 0.838759689922 
epoch 17, loss: 5.581602, error rate: 0.838759689922 
epoch 18, loss: 5.499961, error rate: 

In [1]:
print 0.1 / 10

0.01
